In [2]:
import pandas as pd

In [3]:
def_df = pd.read_csv("/Users/alyssaliu/Desktop/StatsM148/Event Definitions.csv")
df = pd.read_csv("/Users/alyssaliu/Desktop/StatsM148/export.csv")

In [4]:
def_df

,event_name,journey_id,event_definition_id,milestone_number,stage
0,application_phone_approved,1,15,1.0,Apply for Credit
1,application_phone_declined,1,16,NaN,Apply for Credit
2,application_phone_pending,1,17,NaN,Apply for Credit
3,application_web_approved,1,12,1.0,Apply for Credit
4,application_web_declined,1,13,NaN,Apply for Credit
5,application_web_pending,1,14,NaN,Apply for Credit
6,application_web_submit,1,3,NaN,Apply for Credit
7,application_web_view,1,19,NaN,Apply for Credit
8,account_activitation,1,29,3.0,Credit Account
9,account_fraud_review,1,37,NaN,Credit Account


In [5]:
print(df.shape)
df.head()

(64911906, 6)


,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22T08:45:22.000Z,1
1,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,2
2,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,3
3,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,4
4,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,5


In [6]:
df['event_timestamp'] = pd.to_datetime(df['event_timestamp']) # convert to pd datetime

## First deal with removing duplicate events

Ben said these could be ignored

In [7]:
# remove duplicate rows - don't factor in journey_steps_until_end because this counts up even for duplicate rows
df_dropped = df.drop(['journey_steps_until_end'], axis = 1)
df_dropped = df_dropped.drop_duplicates()
# df_dropped = df.drop(['journey_steps_until_end'], axis = 1)
df_dropped.head(10)

,customer_id,account_id,ed_id,event_name,event_timestamp
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00
14,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00
15,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00
16,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00
21,-784961211,1773350293,19,application_web_view,2023-08-29 16:01:06+00:00
24,15849251,383997507,4,browse_products,2021-11-04 14:11:15+00:00
25,15849251,383997507,4,browse_products,2021-11-04 14:11:29+00:00
26,15849251,383997507,4,browse_products,2021-11-04 14:12:10+00:00
27,15849251,383997507,4,browse_products,2021-11-04 14:12:21+00:00


In [8]:
print(df_dropped.shape)
df.shape[0] - df_dropped.shape[0] # number of duplicate rows removed ~8 million

(56853035, 5)


8058871

In [9]:
df_dropped['journey_steps_until_end'] = df_dropped.groupby(['customer_id', 'account_id']).cumcount() + 1 # add journey_steps_until_end back in

In [10]:
df_dropped.reset_index(drop=True, inplace=True)
df_dropped.head(10)

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5
5,-784961211,1773350293,19,application_web_view,2023-08-29 16:01:06+00:00,6
6,15849251,383997507,4,browse_products,2021-11-04 14:11:15+00:00,1
7,15849251,383997507,4,browse_products,2021-11-04 14:11:29+00:00,2
8,15849251,383997507,4,browse_products,2021-11-04 14:12:10+00:00,3
9,15849251,383997507,4,browse_products,2021-11-04 14:12:21+00:00,4


In [11]:
df_dropped.dtypes

customer_id                              int64
account_id                               int64
ed_id                                    int64
event_name                              object
event_timestamp            datetime64[ns, UTC]
journey_steps_until_end                  int64
dtype: object

## Look into removing incomplete customers

Determining whether a customer is unsuccessful (quit) or incomplete (the data was just cut off, so we don't know if they ended up having a successful journey or not) 

according to Ben: it is a bit tricky to know if it’s a ‘quit’ or an ‘incomplete’.  What we typically look for is 60+ days since an approval, after that point we have VERY few people order if they haven’t yet.  Here’s our activation curve as of late as an example:

![Image Description](cum_activation.png)

I will create a dataframe for customers who have had 70+ days since activation - to weed out incomplete customers

In [12]:
# get count of account_activations for each customer/account pair
activation_counts = df_dropped.groupby(['customer_id', 'account_id'])['event_name'].apply(lambda x: x.str.count('account_activitation').sum())
activation_counts

customer_id  account_id 
-2147483541   2059949086    0
-2147481037  -1245770865    0
-2147474335  -1929965268    1
-2147474305   1098145658    0
-2147472167   889210381     1
                           ..
 2147480051  -148210516     0
 2147480182  -1286764612    0
 2147480920   496493954     0
 2147482120  -488077144     0
 2147483471   1556733004    0
Name: event_name, Length: 1735767, dtype: int64

In [13]:
# get count of account_activation occurrences
activation_counts.value_counts()

event_name
0    1319326
1     414268
2       2013
3        160
Name: count, dtype: int64

In [14]:
# just get those with multiple activations
multiple_activations = activation_counts[activation_counts > 1]
multiple_activations.value_counts()

event_name
2    2013
3     160
Name: count, dtype: int64

In [15]:
mult_act_cust = list(multiple_activations.index) # list of tuples of (customer_id, account_id) with multiple activations

# create the filter for dataframe rows that correspond to customers with multiple activations
filt = ~df_dropped.apply(lambda row: (row['customer_id'], row['account_id']) in mult_act_cust, axis=1)
df_filtered = df_dropped[filt] # remove customers with multiple activations

In [16]:
df_filtered

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5
...,...,...,...,...,...,...
56853030,-865005459,-983311387,29,account_activitation,2021-05-14 00:00:00+00:00,39
56853031,-865005459,-983311387,5,view_cart,2021-05-15 09:27:47+00:00,40
56853032,-865005459,-983311387,24,campaignemail_clicked,2021-05-15 14:27:33+00:00,41
56853033,-865005459,-983311387,27,account_downpaymentcleared,2021-05-16 00:00:00+00:00,42


#### Now check account_activation counts - should all be either 1 or 0

In [18]:
# now make sure all remaining customer/account combinations have only 0 or 1 account_activations
df_filtered.groupby(['customer_id', 'account_id'])['event_name'].apply(lambda x: x.str.count('account_activitation').sum()).value_counts()

event_name
0    1319326
1     414268
Name: count, dtype: int64

#### It seems that multiple account activations just indicates multiple journeys under the same customer_id and account_id which is interesting

Because such a small portion of data was removed by removing customer with multiple activations, we will proceed with df_filtered

In [19]:
df_dropped[['customer_id', 'account_id']].drop_duplicates().shape # 1735767 unique combinations of customer/account id

(1735767, 2)

In [20]:
print(2173 / 1735767) # 0.001% of customer/account combinations would be removed if we removed those with multiple account activations
238751 / df_dropped.shape[0] # 0.4% of dataset would be dropped if we removed customers with multiple activations

0.0012518961358292905


0.004199441595334356

Because such a small portion of the dataset has multiple activations, we will drop these rows for simplicity and for the sake of keeping the data uniform.

We will proceed with attempting to remove "incomplete" customers in the data

In [21]:
df_filtered

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5
...,...,...,...,...,...,...
56853030,-865005459,-983311387,29,account_activitation,2021-05-14 00:00:00+00:00,39
56853031,-865005459,-983311387,5,view_cart,2021-05-15 09:27:47+00:00,40
56853032,-865005459,-983311387,24,campaignemail_clicked,2021-05-15 14:27:33+00:00,41
56853033,-865005459,-983311387,27,account_downpaymentcleared,2021-05-16 00:00:00+00:00,42


### Before proceeding with turning data into wide format, first add 'stage' column based on definition df

In [22]:
stage_dict = {'Apply for Credit' : 1, 'Credit Account' : 2, 'Discover' : 3, 'Downpayment' : 4, 'First Purchase' : 5, 
              'Order Shipped' : 6, 'Prospecting' : 7} # create dict to map stage to int
def_df['stage_int'] = def_df['stage'].map(stage_dict)

In [25]:
def_df.sort_values('event_definition_id')


,event_name,journey_id,event_definition_id,milestone_number,stage,stage_int
10,campaign_click,1,2,NaN,Discover,3
6,application_web_submit,1,3,NaN,Apply for Credit,1
21,browse_products,1,4,NaN,First Purchase,5
24,view_cart,1,5,NaN,First Purchase,5
20,begin_checkout,1,6,NaN,First Purchase,5
23,place_order_web,1,7,2.0,First Purchase,5
17,place_downpayment,1,8,4.0,Downpayment,4
11,customer_requested_catalog_digital,1,9,NaN,Discover,3
12,fingerhut_university,1,10,NaN,Discover,3
19,add_to_cart,1,11,NaN,First Purchase,5


In [32]:
event_dict = def_df.set_index('event_definition_id')['stage'].to_dict()

# these are missing, so manually assign them
event_dict[1] = 'Promotion Created'
event_dict[24] = 'Campaignemail Clicked'

In [57]:
event_dict

{15: 'Apply for Credit',
 16: 'Apply for Credit',
 17: 'Apply for Credit',
 12: 'Apply for Credit',
 13: 'Apply for Credit',
 14: 'Apply for Credit',
 3: 'Apply for Credit',
 19: 'Apply for Credit',
 29: 'Credit Account',
 37: 'Credit Account',
 2: 'Discover',
 9: 'Discover',
 10: 'Discover',
 22: 'Discover',
 23: 'Discover',
 27: 'Downpayment',
 26: 'Downpayment',
 8: 'Downpayment',
 25: 'Downpayment',
 11: 'First Purchase',
 6: 'First Purchase',
 4: 'First Purchase',
 18: 'First Purchase',
 7: 'First Purchase',
 5: 'First Purchase',
 28: 'Order Shipped',
 20: 'Prospecting',
 21: 'Prospecting',
 1: 'Promotion Created',
 24: 'Campaignemail Clicked'}

In [33]:
df_filtered['stage'] = df_filtered['ed_id'].map(event_dict)
df_filtered

/var/folders/x3/sb0g2l5d0bx050hnt31710zw0000gn/T/ipykernel_4719/3952731588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['stage'] = df_filtered['ed_id'].map(event_dict)


,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end,stage
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1,Apply for Credit
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2,Apply for Credit
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3,Apply for Credit
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4,Discover
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5,Apply for Credit
...,...,...,...,...,...,...,...
56853030,-865005459,-983311387,29,account_activitation,2021-05-14 00:00:00+00:00,39,Credit Account
56853031,-865005459,-983311387,5,view_cart,2021-05-15 09:27:47+00:00,40,First Purchase
56853032,-865005459,-983311387,24,campaignemail_clicked,2021-05-15 14:27:33+00:00,41,Campaignemail Clicked
56853033,-865005459,-983311387,27,account_downpaymentcleared,2021-05-16 00:00:00+00:00,42,Downpayment


In [56]:
df_filtered['stage'].isna().sum()

0

## Now onto data cleaning - turning data into wide format

In [34]:
# Group by 'customer_id' and aggregate the other columns into lists
df_grouped_cust_acct = df_filtered.groupby(['customer_id', 'account_id']).agg({
    'ed_id': list,
    'event_name': list,
    'event_timestamp': list,
    'journey_steps_until_end': list,
    'stage' : list
}).reset_index()

## add rows on account activation status

In [35]:
# 1 if row activated account, 0 if not
df_grouped_cust_acct['account_activation'] = df_grouped_cust_acct['event_name'].apply(lambda x: 1 if 'account_activitation' in x else 0)

# 1 if row placed order, 0 if did not
df_grouped_cust_acct['place_order'] = df_grouped_cust_acct['event_name'].apply(lambda x: 1 if any([i in x for i in ['place_order_web', 'place_order_phone']]) else 0)

In [37]:
activated_no_order = df_grouped_cust_acct[(df_grouped_cust_acct['account_activation'] == 1) & (df_grouped_cust_acct['place_order'] == 0)][['customer_id', 'account_id']]
activated_no_order

,customer_id,account_id
69,-2147329419,-1587820208
114,-2147234107,-1102522038
175,-2147056730,-1533025295
199,-2146996802,83798101
204,-2146993185,-118642405
...,...,...
1733363,2146935684,1733407043
1733367,2146940656,611604413
1733379,2146976771,1091209054
1733438,2147102719,-1080561934


### Now proceed with removing incomplete customers - those who activated within the last 60 days

In [44]:
cutoff_date = df_dropped['event_timestamp'].max() # extract last date in dataset

activation_events = df_filtered[df_filtered['event_name'] == 'account_activitation'] # get account activations

# Merge to keep only activation events where the order wasn't shipped
df_merged = activation_events.merge(activated_no_order, on=['customer_id', 'account_id'])

# Group by 'customer_id' and 'account_id' to find the latest activation timestamp for each
activation_times = df_merged.groupby(['customer_id', 'account_id'])['event_timestamp'].max().reset_index()

# Calculate the number of days since activation for each customer/account pair
activation_times['days_since_activation'] = (cutoff_date - activation_times['event_timestamp']).dt.days

# Filter out rows where 'days_since_activation' is greater than 60
incomplete_cust_df = activation_times[activation_times['days_since_activation'] <= 60]

In [59]:
cutoff_date

Timestamp('2023-09-20 12:29:58+0000', tz='UTC')

In [45]:
incomplete_cust_df

,customer_id,account_id,event_timestamp,days_since_activation
39,-2144776994,-151636995,2023-09-08 00:00:00+00:00,12
54,-2144107130,252180205,2023-09-13 00:00:00+00:00,7
66,-2143223728,939055895,2023-08-09 00:00:00+00:00,42
75,-2142610706,-1264783980,2023-07-25 00:00:00+00:00,57
82,-2142222132,1186915905,2023-09-03 00:00:00+00:00,17
...,...,...,...,...
57900,2143801624,1066585907,2023-09-08 00:00:00+00:00,12
57911,2144649474,878261980,2023-09-17 00:00:00+00:00,3
57914,2144844060,958270163,2023-08-14 00:00:00+00:00,37
57917,2145083814,754007432,2023-08-18 00:00:00+00:00,33


In [52]:
incomplete_cust = set(zip(incomplete_cust_df['customer_id'], incomplete_cust_df['account_id']))
cleaned_df = df_grouped_cust_acct[~df_grouped_cust_acct.apply(lambda row: (row['customer_id'], row['account_id']) in incomplete_cust, axis=1)]
cleaned_df

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end,stage,account_activation,place_order
0,-2147483541,2059949086,"[12, 5, 4, 11, 5, 6, 1, 4, 11, 4, 11, 5, 6, 5,...","[application_web_approved, view_cart, browse_p...","[2021-10-05 08:11:26+00:00, 2021-10-05 08:17:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Apply for Credit, First Purchase, First Purch...",0,0
1,-2147481037,-1245770865,"[12, 1, 2, 5, 11, 4, 6]","[application_web_approved, promotion_created, ...","[2023-08-26 12:08:34+00:00, 2023-08-26 17:18:4...","[1, 2, 3, 4, 5, 6, 7]","[Apply for Credit, Promotion Created, Discover...",0,0
2,-2147474335,-1929965268,"[2, 12, 19, 3, 4, 4, 19, 4, 1, 11, 4, 5, 11, 4...","[campaign_click, application_web_approved, app...","[2023-04-14 01:42:50+00:00, 2023-04-14 19:42:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Discover, Apply for Credit, Apply for Credit,...",1,1
3,-2147474305,1098145658,"[6, 19, 19, 19, 19, 19, 19, 19, 1, 4, 19, 19, ...","[begin_checkout, application_web_view, applica...","[2021-11-28 22:27:11+00:00, 2021-11-28 22:28:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[First Purchase, Apply for Credit, Apply for C...",0,0
4,-2147472167,889210381,"[12, 19, 3, 4, 4, 1, 1, 11, 5, 4, 11, 5, 4, 6,...","[application_web_approved, application_web_vie...","[2023-02-19 08:23:54+00:00, 2023-02-19 14:21:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Apply for Credit, Apply for Credit, Apply for...",1,1
...,...,...,...,...,...,...,...,...,...
1733589,2147480051,-148210516,"[21, 22, 2, 12, 24, 1, 21, 1, 1, 21, 1, 1, 1, 1]","[catalog_(mail), pre-application_(3rd_party_af...","[2020-12-18 00:00:00+00:00, 2021-01-23 06:00:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]","[Prospecting, Discover, Discover, Apply for Cr...",0,0
1733590,2147480182,-1286764612,"[12, 2, 24, 4, 24, 24, 24, 24, 24]","[application_web_approved, campaign_click, cam...","[2023-06-22 09:44:49+00:00, 2023-06-22 15:44:4...","[1, 2, 3, 4, 5, 6, 7, 8, 9]","[Apply for Credit, Discover, Campaignemail Cli...",0,0
1733591,2147480920,496493954,"[2, 19, 19, 19, 19, 19, 19, 19, 3, 19, 12, 4, ...","[campaign_click, application_web_view, applica...","[2022-05-21 06:00:00+00:00, 2022-05-21 23:30:3...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Discover, Apply for Credit, Apply for Credit,...",0,0
1733592,2147482120,-488077144,"[12, 19, 2]","[application_web_approved, application_web_vie...","[2023-07-30 15:22:56+00:00, 2023-07-30 20:18:5...","[1, 2, 3]","[Apply for Credit, Apply for Credit, Discover]",0,0


In [54]:
cleaned_df.to_pickle("/Users/alyssaliu/Desktop/StatsM148/cleaned_wide_format_data.pkl")

## Old wide format data

In [28]:
# Group by 'customer_id' and aggregate the other columns into lists
df_grouped = df_dropped.groupby('customer_id').agg({
    'account_id': list,
    'ed_id': list,
    'event_name': list,
    'event_timestamp': list,
    'journey_steps_until_end': list,
}).reset_index()

In [29]:
df_grouped.head(2)

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-2147483541,"[2059949086, 2059949086, 2059949086, 205994908...","[12, 5, 4, 11, 5, 6, 1, 4, 11, 4, 11, 5, 6, 5,...","[application_web_approved, view_cart, browse_p...","[2021-10-05 08:11:26+00:00, 2021-10-05 08:17:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,-2147481037,"[-1245770865, -1245770865, -1245770865, -12457...","[12, 1, 2, 5, 11, 4, 6]","[application_web_approved, promotion_created, ...","[2023-08-26 12:08:34+00:00, 2023-08-26 17:18:4...","[1, 2, 3, 4, 5, 6, 7]"


In [30]:
df_grouped.shape

(1665688, 6)

In [31]:
df_grouped['account_id'] = df_grouped['account_id'].apply(lambda x: list(set(x)))

In [32]:
df_grouped.head()

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-2147483541,[2059949086],"[12, 5, 4, 11, 5, 6, 1, 4, 11, 4, 11, 5, 6, 5,...","[application_web_approved, view_cart, browse_p...","[2021-10-05 08:11:26+00:00, 2021-10-05 08:17:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,-2147481037,[-1245770865],"[12, 1, 2, 5, 11, 4, 6]","[application_web_approved, promotion_created, ...","[2023-08-26 12:08:34+00:00, 2023-08-26 17:18:4...","[1, 2, 3, 4, 5, 6, 7]"
2,-2147474335,[-1929965268],"[2, 12, 19, 3, 4, 4, 19, 4, 1, 11, 4, 5, 11, 4...","[campaign_click, application_web_approved, app...","[2023-04-14 01:42:50+00:00, 2023-04-14 19:42:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,-2147474305,[1098145658],"[6, 19, 19, 19, 19, 19, 19, 19, 1, 4, 19, 19, ...","[begin_checkout, application_web_view, applica...","[2021-11-28 22:27:11+00:00, 2021-11-28 22:28:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,-2147472167,[889210381],"[12, 19, 3, 4, 4, 1, 1, 11, 5, 4, 11, 5, 4, 6,...","[application_web_approved, application_web_vie...","[2023-02-19 08:23:54+00:00, 2023-02-19 14:21:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [33]:
df_grouped.to_pickle("wide_format_data_no_duplicates.pkl")

In [12]:
# df_grouped.to_csv('wide_format_df.csv')
# this doesn't preserve lists in the columns, turns them into strings